In [99]:
import json, re, pickle
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from gensim.models.word2vec import Word2Vec

with open('static/data/train-v2.0.json', 'r', encoding='utf-8') as file:
# with open('test.json', 'r', encoding='utf-8') as file:
    data = json.load(file)


In [103]:
df_dict, text_df_dict = {}, {}
a_id = c_id = q_id = 0

for t_id, d in enumerate(data['data']):
    for paragraph in d['paragraphs']:
        for qa in paragraph['qas']:
            col = 'answers' if not qa['is_impossible'] else 'plausible_answers'
            for a in qa[col]:
                text_df_dict[a_id] = {
                    'title': d['title'],
                    'question': qa['question'],
                    'context': paragraph['context'],
                    'answer': a['text'] if not qa['is_impossible'] else np.nan,
                    'plausible_answer': np.nan if not qa['is_impossible'] else a['text']
                }
                
                df_dict[a_id] = {
                    'title': t_id,
                    'question': q_id,
                    'context': c_id,
                    'context': c_id,
                    'answer': a['answer_start'] if not qa['is_impossible'] else -1,
                    'plausible_answer': -1 if not qa['is_impossible'] else a['answer_start']
                }
                
                a_id += 1
            q_id += 1
        c_id += 1

df = pd.DataFrame(df_dict).T.astype(int)
text_df = pd.DataFrame(text_df_dict).T
df

,title,question,context,answer,plausible_answer
0,0,0,0,269,-1
1,0,1,0,207,-1
2,0,2,0,526,-1
3,0,3,0,166,-1
4,0,4,0,276,-1
...,...,...,...,...,...
130314,441,130314,19034,-1,485
130315,441,130315,19034,-1,327
130316,441,130316,19034,-1,350
130317,441,130317,19034,-1,529


In [101]:
title = df['title'].to_dict()
question = df['question'].to_dict()
context = df['context'].to_dict()
answer = df['answer'].to_dict()

In [77]:

df = pd.json_normalize(df['qas'].sum(), sep='_', ).join(col)
df

,question,id,answers,is_impossible,plausible_answers,title
0,When did Beyonce start becoming popular?,56be85543aeaaa14008c9063,"[{'text': 'in the late 1990s', 'answer_start':...",False,NaN,Beyoncé
1,What areas did Beyonce compete in when she was...,56be85543aeaaa14008c9065,"[{'text': 'singing and dancing', 'answer_start...",False,NaN,Beyoncé
2,When did Beyonce leave Destiny's Child and bec...,56be85543aeaaa14008c9066,"[{'text': '2003', 'answer_start': 526}]",False,NaN,Beyoncé
3,In what city and state did Beyonce grow up?,56bf6b0f3aeaaa14008c9601,"[{'text': 'Houston, Texas', 'answer_start': 166}]",False,NaN,Beyoncé
4,In which decade did Beyonce become famous?,56bf6b0f3aeaaa14008c9602,"[{'text': 'late 1990s', 'answer_start': 276}]",False,NaN,Beyoncé
...,...,...,...,...,...,...
130314,Physics has broadly agreed on the definition o...,5a7e070b70df9f001a875439,[],True,"[{'text': 'matter', 'answer_start': 485}]",Matter
130315,Who coined the term partonic matter?,5a7e070b70df9f001a87543a,[],True,"[{'text': 'Alfvén', 'answer_start': 327}]",Matter
130316,What is another name for anti-matter?,5a7e070b70df9f001a87543b,[],True,"[{'text': 'Gk. common matter', 'answer_start':...",Matter
130317,Matter usually does not need to be used in con...,5a7e070b70df9f001a87543c,[],True,"[{'text': 'a specifying modifier', 'answer_sta...",Matter


In [14]:

# Set "title" and "id" as index columns
df.set_index(['title', 'id'], inplace=True)
df

question  \
title id                                                                            
IPod  56cc55856d243a140015ef0a                   Which company produces the iPod?   
      56cc55856d243a140015ef0b               When was the original iPod released?   
      56cc55856d243a140015ef0c  How many different types of iPod are currently...   
      56cc55856d243a140015ef0d                   What kind of device is the iPod?   
      56cc55856d243a140015ef0e        The iPod Touch uses what kind of interface?   
...                                                                           ...   
NaN   5a8dc46bdf8bba001a0f9be7  Who wrote and provided high-definition for a T...   
      5a8dc46bdf8bba001a0f9be8  The Twilight Princess game was only available ...   
      5a8dc46bdf8bba001a0f9be9  Who published the high-definition comic book s...   
      5a8dc46bdf8bba001a0f9bea  What kind of device can access the high-defini...   
      5a8dc46bdf8bba001a0f9beb  When was a high-definition manga based on Twil...   

                                                                          answers  \
title id                                                                            
IPod  56cc55856d243a140015ef0a           [{'text': 'Apple', 'answer_start': 105}]   
      56cc55856d243a140015ef0b  [{'text': 'October 23, 2001', 'answer_start': ...   
      56cc55856d243a140015ef0c           [{'text': 'three', 'answer_start': 302}]   
      56cc55856d243a140015ef0d  [{'text': 'portable media players', 'answer_st...   
      56cc55856d243a140015ef0e     [{'text': 'touchscreen', 'answer_start': 400}]   
...                                                                           ...   
NaN   5a8dc46bdf8bba001a0f9be7                                                 []   
      5a8dc46bdf8bba001a0f9be8                                                 []   
      5a8dc46bdf8bba001a0f9be9                                                 []   
      5a8dc46bdf8bba001a0f9bea                                                 []   
      5a8dc46bdf8bba001a0f9beb                                                 []   

                                is_impossible  \
title id                                        
IPod  56cc55856d243a140015ef0a          False   
      56cc55856d243a140015ef0b          False   
      56cc55856d243a140015ef0c          False   
      56cc55856d243a140015ef0d          False   
      56cc55856d243a140015ef0e          False   
...                                       ...   
NaN   5a8dc46bdf8bba001a0f9be7           True   
      5a8dc46bdf8bba001a0f9be8           True   
      5a8dc46bdf8bba001a0f9be9           True   
      5a8dc46bdf8bba001a0f9bea           True   
      5a8dc46bdf8bba001a0f9beb           True   

                                                                plausible_answers  
title id                                                                           
IPod  56cc55856d243a140015ef0a                                                NaN  
      56cc55856d243a140015ef0b                                                NaN  
      56cc55856d243a140015ef0c                                                NaN  
      56cc55856d243a140015ef0d                                                NaN  
      56cc55856d243a140015ef0e                                                NaN  
...                                                                           ...  
NaN   5a8dc46bdf8bba001a0f9be7   [{'text': 'Akira Himekawa', 'answer_start': 85}]  
      5a8dc46bdf8bba001a0f9be8             [{'text': 'Japan', 'answer_start': 2}]  
      5a8dc46bdf8bba001a0f9be9      [{'text': 'Shogakukan', 'answer_start': 186}]  
      5a8dc46bdf8bba001a0f9bea          [{'text': 'mobile', 'answer_start': 208}]  
      5a8dc46bdf8bba001a0f9beb  [{'text': 'February 8, 2016', 'answer_start': ...  

[701 rows x 4 columns]

In [ ]:

# Display the DataFrame
df.head()


In [22]:
df.loc[("IPod", "56cc55856d243a140015ef0a")]
a = ""
for i in df.index:
    if i[0] != a:
        print(i[0])
        a = i[0]

IPod
The_Legend_of_Zelda:_Twilight_Princess
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [104]:
result_dict = {}
for d in data['data']:
    for paragraph in d['paragraphs']:
        for qa in paragraph['qas']:
            col = 'answers' if not qa['is_impossible'] else 'plausible_answers'
            for a in qa[col]:
                result_dict[a['answer_start']] = a['text']

result_dict

{269: 'expatriates',
 207: 'cultural traditions based on material',
 526: 'predators',
 166: '30%',
 276: 'Petite Anse',
 320: 'Tags',
 505: 'the apparent asymmetry of matter and antimatter',
 360: 'Buddhist and Hindu influence',
 290: 'Nordic',
 590: 'public housing projects',
 369: "a trophy hunters' expenditures",
 565: 'four',
 260: 'Shockoe Bottom',
 586: 'the Russian Empire or its vassal state, the Emirate of Bukhara',
 180: 'license to hunt migratory birds',
 406: 'the Science Department',
 48: 'electromagnetic',
 95: 'Kasthamandap Mahanagar',
 466: 'the energy–momentum',
 104: 'to exclude other bodies',
 935: 'Uthman ibn Affan',
 985: 'North Carolina',
 736: 'Henry Cabot Lodge',
 18: 'President Franklin D. Roosevelt',
 970: '1990s',
 393: 'viruses',
 445: 'plasmas, superfluids, supersolids, Bose–Einstein condensates',
 552: 'by removing territorial bounds that would otherwise be established',
 303: 'Matter',
 204: 'gravitational effects',
 330: '13,040',
 578: 'A senator',
 152

In [ ]:
def tokenize(df: pd.DataFrame) -> None:
    low = lambda x: x.lower()
    html = lambda x: BeautifulSoup(x, "html.parser").get_text()
    dots = lambda x: re.sub(r'\.{3,}', "...", x)
    dash = lambda x: re.sub(r'-{2,}', "--", x)
    space = lambda x: re.sub(r'\s+', ' ', x)
    quotes = lambda x: re.sub(r'[\'"]\b([^\'"]{1,20})\b[\'"]', r'\1', x)
    punct = lambda x: re.sub(r'(\.{3}|-{2}|[.,:;!?\'])', r'<\1>', x)
    split = lambda x: re.findall(r'\b\w+\b|<[.,:;!?\']>|<...>|<-->', x)
    
    # TODO remove string.punctuation
    
    pipeline = lambda x: split(punct(quotes(space(dash(dots(low(html(x))))))))
    
    df['tokenized'] = df['input_sentence'].apply(lambda x: pipeline(x)+[EOS])

    df['tokenized'].to_pickle("tokenized.pickle", protocol=pickle.HIGHEST_PROTOCOL)
    
    return df
    
tokenize(title)
tokenize(question)
tokenize(context)
tokenize(answer)